# **Module 2: Hypothesis Testing for Proportion**


#Categorical Variable vs Categorical Variable
___

In [ ]:
#import numpy and pandas library
import numpy as np 
import pandas as pd

___

## **a. One Sample Z-Test**

Menguji apakah proporsi dari suatu kelompok A sama dengan asumsi proporsi teoritis yang diinginkan.

In [ ]:
# read dataset 
df = pd.read_csv('bestsellers with categories.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


Kita ingin membuktikan apakah proporsi buku Fiction dari seluruh genre adalah 50%

Hypothesis:
- Ho: Proporsi Fiction = 0.5
- Ha: Proporsi Fiction ≠ 0.5

Sigficance level (alpha) = 0.05

In [ ]:
# proporsi buku Fiction pada sample

jml_fiction = len(df[df['Genre']=='Fiction'])
jml_all = len(df['Genre'])

prop_sample = jml_fiction / jml_all
prop_sample 

0.43636363636363634

In [ ]:
#import library proportions ztest
from statsmodels.stats.proportion import proportions_ztest 

In [ ]:
stat, pvalue = proportions_ztest(
    count= jml_fiction,     # jumlah buku fiction
    nobs= jml_all,          # jumlah seluruh buku (sample)
    value= 0.5              # proporsi teoritis (Ho)
)

pvalue

0.0026186572865008824

In [ ]:
# conclusion

if pvalue > 0.05:
    print(f'''
    Pvalue = {pvalue}. Pvalue > 0.05. Gagal menolak Ho.
    Kita TIDAK punya cukup bukti untuk mengatakan bahwa proporsi buku Fiction tidak sama dengan 50%.
    (Proporsinya TIDAK berbeda signifikan)
    ''')

else:
    print(f'''
    Pvalue = {pvalue}. Pvalue <= 0.05. Berhasil menolak Ho.
    Kita punya cukup bukti untuk mengatakan bahwa proporsi buku Fiction tidak sama dengan 50%.
    (Proporsinya berbeda signifikan)
    ''')    


    Pvalue = 0.0026186572865008824. Pvalue <= 0.05. Berhasil menolak Ho.
    Kita punya cukup bukti untuk mengatakan bahwa proporsi buku Fiction tidak sama dengan 50%.
    (Proporsinya berbeda signifikan)
    


43% (proporsi sample) berbeda signifikan dari 50% (proporsi teoritis)
___

## **b. Two Sample Z-Test**

Membandingkan proporsi dari 2 kelompok berbeda

Kasus: A/B testing

Sebuah e-commerce membuat design UI baru. Kemudian dilakukan pengujian dengan 1000 user diberi design lama (control group) dan 1000 user diberi design baru (treatment group). Apakah konversi atau transaksi meningkat dengan design baru?

- Ho: transaksi kelompok design layout baru = layout lama
- Ha: transaksi kelompok design layout baru > layout lama


In [ ]:
# buat data dummy
np.random.seed(0)

visitor_data_before = np.random.binomial(1, 0.5, 1000)
visitor_data_after = np.random.binomial(1, 0.7, 1000)

visitor_data_before = np.where(visitor_data_before ==1, 'buying', 'no buying')
visitor_data_after = np.where(visitor_data_after ==1, 'buying', 'no buying')

df_old = pd.DataFrame({'layout':'old layout', 'conversion': visitor_data_before})
df_new = pd.DataFrame({'layout':'new layout', 'conversion': visitor_data_after})

In [ ]:
df_old['conversion'].value_counts()
# conversion rate (proporsi transaksi) pada kelompok old layout = 48.3%

no buying    517
buying       483
Name: conversion, dtype: int64

In [ ]:
df_new['conversion'].value_counts()
# conversion rate (proporsi transaksi) pada kelompok new layout = 66.4%

buying       664
no buying    336
Name: conversion, dtype: int64

In [ ]:
# menggabungkan dataframe
df_all = pd.concat([df_old, df_new]).reset_index(drop=True)
df_all

,layout,conversion
0,old layout,buying
1,old layout,buying
2,old layout,buying
3,old layout,buying
4,old layout,no buying
...,...,...
1995,new layout,no buying
1996,new layout,no buying
1997,new layout,buying
1998,new layout,buying


In [ ]:
# jumlah pembelian
df_pivot = df_all.pivot_table(
    index= 'layout',
    columns= 'conversion',
    values= 'conversion',
    aggfunc= len
)

df_pivot

conversion,buying,no buying
layout,,
new layout,664,336
old layout,483,517


In [ ]:
# jumlah pembelian old layout
n_buying_old = df_pivot.loc['old layout', 'buying']
n_buying_old 

483

In [ ]:
# jumlah pembelian new layout
n_buying_new = df_pivot.loc['new layout', 'buying']
n_buying_new 

664

In [ ]:
# jumlah seluruh kelompok old
n_old = df_pivot.sum(axis=1)['old layout']
n_old 

1000

In [ ]:
# jumlah seluruh kelompok new
n_new = df_pivot.sum(axis=1)['new layout']
n_new 

1000

In [ ]:
stat, pvalue = proportions_ztest(
    count= [n_buying_new, n_buying_old],   # jml pembelian di kelompok new dan kelompok old
    nobs= [n_new, n_old],                  # jumlah seluruh kelompok new dan jumlah seluruh kelompok old
    alternative='larger'                   # Ha (lebih besar)
)

pvalue

1.3789563948761732e-16

In [ ]:
# conclusion

if pvalue > 0.05:
    print(f'''
    Pvalue = {pvalue}. Pvalue > 0.05. Gagal menolak Ho.
    Kita TIDAK punya cukup bukti untuk mengatakan bahwa proporsi pembelian pada kelompok New 
    lebih besar dari proporsi pembelian pada kelompok Old.
    (Proporsinya TIDAK berbeda signifikan)
    ''')

else:
    print(f'''
    Pvalue = {pvalue}. Pvalue <= 0.05. Berhasil menolak Ho.
    Kita punya cukup bukti untuk mengatakan bahwa proporsi pembelian pada kelompok New 
    lebih besar dari proporsi pembelian pada kelompok Old.
    (Proporsinya berbeda signifikan)
    ''')    


    Pvalue = 1.3789563948761732e-16. Pvalue <= 0.05. Berhasil menolak Ho.
    Kita punya cukup bukti untuk mengatakan bahwa proporsi pembelian pada kelompok New 
    lebih besar dari proporsi pembelian pada kelompok Old.
    (Proporsinya berbeda signifikan)
    


___

## **c. Chi-Square Test**

Menguji independency/keterkaitan antara 2 categorical variable.

Di bawah ini adalah crosstab dari 2 kolom Gender dan Fruit. 
Kita akan menguji apakah Gender terkait dengan pilihan buah yg akan dibeli.

Hypothesis:
- Ho: Gender dan Fruit tidak terkait/independent (proporsinya sama)
- Ha: Gender dan Fruit terkait/dependent (proporsinya berbeda signifikan)

In [ ]:
# membuat dataframe baru
df_c = pd.DataFrame(
    data= [[207,282,241], [234,242,232]],
    columns= ['Banana', 'Mango', 'Pinaple'],
    index = ['Male', 'Female']
)

df_c

,Banana,Mango,Pinaple
Male,207,282,241
Female,234,242,232


In [ ]:
#import chi2_contingency library
from scipy.stats import chi2_contingency

In [ ]:
# input dari function chi2_contingency ini berupa crosstab
stat, pvalue, dof, expexted = chi2_contingency(df_c)
pvalue

0.1031971404730939

In [ ]:
# conclusion

if pvalue > 0.05:
    print(f'''
    Pvalue = {pvalue}. Pvalue > 0.05. Gagal menolak Ho.
    Kita TIDAK punya cukup bukti untuk mengatakan bahwa Gender dan Fruit terkait/dependent.
    (Proporsinya TIDAK berbeda signifikan)
    ''')

else:
    print(f'''
    Pvalue = {pvalue}. Pvalue <= 0.05. Berhasil menolak Ho.
    Kita punya cukup bukti untuk mengatakan bahwa Gender dan Fruit terkait/dependent.
    (Proporsinya berbeda signifikan)
    ''')    


    Pvalue = 0.1031971404730939. Pvalue > 0.05. Gagal menolak Ho.
    Kita TIDAK punya cukup bukti untuk mengatakan bahwa Gender dan Fruit terkait/dependent.
    (Proporsinya TIDAK berbeda signifikan)
    


___